<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/32a_langchain_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ต่อไปเราจะมาเรียนรู้ LangChain

# 0. Magic Commands
ให้ใส่ไว้บนสุดทุก Notebook เป็นการสั่งให้ Notebook ก่อนรัน ให้รีโหลด Library ภายนอกที่เรา import ไว้ใหม่โดยอัตโนมัติ

และให้พล็อตกราฟ matplotlib ใน Output ของ cell แบบ code ได้เลย

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

สั่งให้ Colab แสดงผลข้อความแบบ Wrap Text

In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# 1. Import Library

ติดตั้ง OpenAI Library ด้วยการ Uncomment Cell ด้านล่างแล้ว Run

In [6]:
# ! pip install openai

# 2. Authentication

ก่อนที่เราจะเรียกใช้งาน ChatGPT, GPT-4 ของ OpenAI ได้ เราต้องลงทะเบียน สมัครใช้งาน และใส่ข้อมูลวิธีชำระเงินให้เรียบร้อยก่อน 

## Sign up

กดปุ่ม Sign up สีเขียว ในลิงค์ด้านล่าง สมัครใช้งาน OpenAI 

https://platform.openai.com/overview

ใส่ข้อมูลบัตรเครดิต ในหน้า Billing ให้เรียบร้อย

https://platform.openai.com/account/billing/payment-methods






## API Key

สร้าง API Key จากลิงค์ด้านล่าง โดยกดปุ่ม Create new secret key ตั้งชื่อ แล้ว Copy secret key เตรียมไว้ใส่ในช่อง OPENAI_API_KEY ด้านล่าง

https://platform.openai.com/account/api-keys


## ตัวอย่าง API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

รัน Cell ด้านล่าง แล้วใส่ secret key ที่ได้มา

บางที Run แล้วค้าง ไม่ขึ้นช่องให้กรอก ให้กดหยุด แล้วลอง Run ใหม่

In [8]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

os.environ["OPENAI_API_KEY"] = token

Enter OPENAI_API_KEY here: ··········


หลังจากทดสอบเสร็จแล้ว แนะนำให้กลับไปลบ Secret Key ในหน้า https://platform.openai.com/account/api-keys ทิ้งไปทุกครั้ง เพื่อความปลอดภัย โดยเราสามารถสร้าง Key ใหม่ได้เรื่อย ๆ ไม่จำกัด

#3. Call API 

## 3.1 เรียกใช้ OpenAI API แบบ ตรง ๆ 

[จาก ep ก่อน Prompt Engineering คืออะไร ](https://www.bualabs.com/archives/4483/what-is-prompt-engineering/) เราได้เรียกใช้ chatgpt ผ่าน API ของ OpenAI ได้โดยตรง

In [11]:
import openai

In [12]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [94]:
country = "Thailand"
language = "English"

prompt31 = f"""Give me 10 reasons why {country} is a good place to retire.
Output in {language}.
"""

In [95]:
print(prompt31)

Give me 10 reasons why Thailand is a good place to retire.
Output in English.



In [96]:
reason31 = get_completion(prompt31)

In [97]:
print(reason31)

1. Affordable cost of living: Thailand offers a low cost of living compared to many Western countries, making it an attractive option for retirees on a budget.

2. Warm climate: With its tropical climate, Thailand offers warm weather year-round, which is ideal for retirees who want to enjoy outdoor activities.

3. High-quality healthcare: Thailand has a well-developed healthcare system, with modern facilities and highly trained medical professionals.

4. Friendly locals: Thai people are known for their warm hospitality and friendly nature, making it easy for retirees to feel at home in the country.

5. Delicious food: Thailand is famous for its delicious cuisine, which is both healthy and affordable.

6. Beautiful scenery: From stunning beaches to lush jungles, Thailand offers retirees a wide range of beautiful scenery to explore.

7. Rich culture: Thailand has a rich cultural heritage, with ancient temples, traditional festivals, and a vibrant arts scene.

8. Easy visa process: Thaila

In [115]:
country = "Singapore"
language = "English with stutter in every sentences"

prompt31 = f"""Give me 10 reasons why {country} is a good place to retire.
Output in {language}.
"""

In [116]:
print(prompt31)

Give me 10 reasons why Singapore is a good place to retire.
Output in English with stutter in every sentences.



In [117]:
reason31 = get_completion(prompt31)
print(reason31)

W-w-well, S-Singapore is a g-g-great place to retire because:

1. It has a s-s-safe and secure environment.
2. The healthcare system is w-w-world-class.
3. The public transportation is efficient and accessible.
4. The c-c-climate is warm and tropical.
5. There are plenty of parks and green spaces to enjoy.
6. The food scene is diverse and delicious.
7. There are many cultural and recreational activities to participate in.
8. The economy is stable and prosperous.
9. The government provides support for retirees.
10. It's a m-m-multicultural society with a welcoming attitude towards foreigners.


## 3.2 เรียกใช้ ChatGPT ผ่าน LangChain

เราสามารถใช้ F-String ในการสร้าง Template เพื่อสร้าง Prompt ส่งไปยัง ChatGPT ได้ไม่มีปัญหา แต่ถ้าเป็น Prompt ที่ซับซ้อนขึ้น มีหลาย Step และมี Output เป็นโครงสร้าง ที่จะนำไปใช้ต่อเช่น JSON การใช้คลาสเฉพาะ เช่น ChatPromptTemplate ของ LangChain สะดวกกว่า

In [72]:
### https://python.langchain.com/en/latest/modules/models/llms/integrations.html

In [73]:
# ! pip install langchain

In [74]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI


In [75]:
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

ChatPromptTemplate

In [118]:
template_string32 = """Give me 10 reasons why {country} is a good place to retire.
Output in {language}."""

In [119]:
prompt_template32 = ChatPromptTemplate.from_template(template_string32)

In [103]:
# prompt_template32

In [104]:
# prompt_template32.messages

In [105]:
# prompt_template32.messages[0]

In [120]:
prompt_template32.messages[0].prompt

PromptTemplate(input_variables=['country', 'language'], output_parser=None, partial_variables={}, template='Give me 10 reasons why {country} is a good place to retire.\nOutput in {language}.', template_format='f-string', validate_template=True)

In [121]:
country_to_retire = "Japanese"   # Not Japan
language_to_write = "Rude English"

prompt32 = prompt_template32.format_messages(
                    country=country_to_retire,
                    language=language_to_write)

In [122]:
print(type(prompt32))
print(prompt32)

<class 'list'>
[HumanMessage(content='Give me 10 reasons why Japanese is a good place to retire.\nOutput in Rude English.', additional_kwargs={}, example=False)]


In [123]:
prompt32[0]

HumanMessage(content='Give me 10 reasons why Japanese is a good place to retire.\nOutput in Rude English.', additional_kwargs={}, example=False)

In [124]:
reasons32 = chat(prompt32)

In [125]:
print(reasons32.content)

1. Shitloads of sushi and ramen to stuff your face with.
2. The public transportation is fucking amazing.
3. The healthcare system is top-notch, so you won't have to worry about dying from some bullshit disease.
4. The crime rate is low as fuck, so you can walk around without getting shanked.
5. The culture is rich as fuck, so you can learn some cool shit.
6. The technology is advanced as fuck, so you can play with some cool gadgets.
7. The scenery is beautiful as fuck, so you can enjoy some breathtaking views.
8. The people are polite as fuck, so you won't have to deal with assholes.
9. The weather is mild as fuck, so you won't have to deal with extreme temperatures.
10. The nightlife is lit as fuck, so you can party like a fucking rockstar.


# 4. Output Format

ใน [ep ก่อน เรื่องระบุรูปแบบโครงสร้างข้อมูลผลลัพท์](https://www.bualabs.com/archives/4703/how-to-prompt-chatgpt-format-output-prompt-engineering-ep-2/) เราได้พูดถึงเรื่อง การกำหนด Format โครงสร้างผลลัพท์ ที่ต้องการใน Prompt เพื่อนำไปใช้ต่อ

## 4.1 ระบุโครงสร้างผลลัพท์ ที่ต้องการใน Prompt โดยตรง

In [142]:
template_string41 = """Give me 10 reasons why {country} is a good place to retire.
Write in {language}. Output in JSON Format with key reason, category"""

In [143]:
prompt_template41 = ChatPromptTemplate.from_template(template_string41)

In [144]:
country_to_retire = "Mexico"   # Not Japan
language_to_write = "Polite English"

prompt41 = prompt_template41.format_messages(
                    country=country_to_retire,
                    language=language_to_write)

In [145]:
reasons41 = chat(prompt41)

In [146]:
print(reasons41.content)

{
    "reasons": [
        {
            "reason": "Affordable cost of living",
            "category": "Financial"
        },
        {
            "reason": "Warm climate year-round",
            "category": "Weather"
        },
        {
            "reason": "Beautiful beaches and scenery",
            "category": "Recreation"
        },
        {
            "reason": "Rich culture and history",
            "category": "Culture"
        },
        {
            "reason": "Excellent healthcare options",
            "category": "Healthcare"
        },
        {
            "reason": "Friendly and welcoming locals",
            "category": "Community"
        },
        {
            "reason": "Easy access to travel within Mexico and to other countries",
            "category": "Travel"
        },
        {
            "reason": "Variety of delicious cuisine",
            "category": "Food"
        },
        {
            "reason": "Opportunities for volunteering and giving back to 

In [147]:
print(type(reasons41.content))

<class 'str'>


## 4.2 ใช้ StructuredOutputParser และ ResponseSchema จาก LangChain

In [148]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [171]:
reason_schema = ResponseSchema(name="reason",
                             description="The reason why it is good to retire in that country")
category_schema = ResponseSchema(name="category",
                                      description="The category of reason")

reasons_schemas = [reason_schema, category_schema]

In [172]:
output_parser = StructuredOutputParser.from_response_schemas(reasons_schemas)

In [173]:
format_instructions = output_parser.get_format_instructions()

In [174]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"reason": string  // The reason why it is good to retire in that country
	"category": string  // The category of reason
}
```


In [175]:
template_string42 = """
Give me 1 reason why {country} is a good place to retire.

Write in {language}. 

{format_instructions}
"""

In [176]:
prompt_template42 = ChatPromptTemplate.from_template(template_string42)

In [177]:
country_to_retire = "Nepal"   # Not Japan
language_to_write = "Funny English"

prompt42 = prompt_template42.format_messages(
                    country=country_to_retire,
                    language=language_to_write, 
                    format_instructions=format_instructions)

In [178]:
print(prompt42[0].content)


Give me 1 reason why Nepal is a good place to retire.

Write in Funny English. 

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"reason": string  // The reason why it is good to retire in that country
	"category": string  // The category of reason
}
```



In [179]:
reasons42 = chat(prompt42)

In [180]:
print(reasons42.content)

```json
{
	"reason": "Because you can finally achieve your lifelong dream of becoming a mountain goat and climb the Himalayas every day!",
	"category": "Humor"
}
```


In [181]:
print(type(reasons42.content))

<class 'str'>


In [182]:
output_dict = output_parser.parse(reasons42.content)

In [183]:
type(output_dict)

dict

In [184]:
print(output_dict)

{'reason': 'Because you can finally achieve your lifelong dream of becoming a mountain goat and climb the Himalayas every day!', 'category': 'Humor'}


In [189]:
print(output_dict["reason"])

Because you can finally achieve your lifelong dream of becoming a mountain goat and climb the Himalayas every day!
